In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which

In [ ]:
import json
import pandas as pd
from datasets import Dataset

# Step 1: Load your different JSON files
json_file1 = "/content/drive/MyDrive/test_vector_db/datasetPH.json"
json_file2 = "/content/drive/MyDrive/test_vector_db/Merged Q& A data.json"
json_file3 = "/content/drive/MyDrive/test_vector_db/structured_to_text_all_years2.json"

# Load each file
with open(json_file1, 'r') as f:
    data1 = json.load(f)

with open(json_file2, 'r') as f:
    data2 = json.load(f)

with open(json_file3, 'r') as f:
    data3 = json.load(f)

# Step 2: Create a function to convert each format to instruction-response pairs
def process_format1(data):
    """Process detailed nested JSON structure into instruction-response pairs"""
    pairs = []

    for paper_id, item in data.items():
        # Build a detailed instruction using available metadata
        instruction = f"Analyze the policy implications of the paper titled '{item['title']}' authored by {item['author']} ({item['publication_year']})."

        # Build a comprehensive response from nested fields
        policy = item.get("policy_practice_implications", {})
        comp_qual = item.get("comparative_and_qualitative_insights", {})

        recommendations = policy.get("recommendations", "No recommendations provided.")
        impl_notes = policy.get("implementation_notes", "")
        limitations = comp_qual.get("limitations", "")
        future_work = comp_qual.get("future_work", "")

        response_parts = [
            f"Recommendations: {recommendations}",
            f"Implementation Notes: {impl_notes}",
        ]
        if limitations:
            response_parts.append(f"Limitations: {limitations}")
        if future_work:
            response_parts.append(f"Future Work: {future_work}")

        response = "\n".join(response_parts)

        pairs.append({
            "instruction": instruction,
            "response": response
        })

    return pairs


def process_format2(data):
    """Process Q&A style JSON dataset with entries of questions and answers"""
    pairs = []

    for item in data.get("entries", []):
        # Get question and answer from the item
        question = item.get('question', '')
        answer = item.get('answer', '')

        # Clean Unicode from both question and answer
        clean_question = question.encode('ascii', 'ignore').decode('ascii')
        clean_answer = answer.encode('ascii', 'ignore').decode('ascii')

        # Replace specific symbols with text equivalents
        clean_question = clean_question.replace("%", " percent")
        clean_answer = clean_answer.replace("%", " percent")
        clean_answer = clean_answer.replace("µg/m³", "ug/m3")

        # Format as instruction and response
        instruction = f"{clean_question}"
        response = f"{clean_answer}"

        pairs.append({"instruction": instruction, "response": response})

    return pairs

def process_format3(data):
    """Process educational statistics from state-year records"""
    pairs = []
    for item in data:
        year = item.get("year", "Unknown Year")
        state = item.get("state", "Unknown State")
        text = item.get("text", "")

        # Create instruction
        instruction = f"Summarize the educational attainment statistics for {state} in {year}."

        # Clean text - remove Unicode and replace % with "percent"
        cleaned_text = text.encode('ascii', 'ignore').decode('ascii')  # Remove all Unicode
        cleaned_text = cleaned_text.replace("!!", " ").replace("Estimate", "")
        cleaned_text = cleaned_text.replace("%", " percent")

        # Format text with better readability without truncation
        cleaned_text = " ".join(cleaned_text.split())

        response = f"Here is a summary of the educational data for {state} in {year}:\n{cleaned_text}"
        pairs.append({"instruction": instruction, "response": response})

    return pairs


# Step 3: Process each file with its corresponding function
pairs1 = process_format1(data1)
pairs2 = process_format2(data2)
pairs3 = process_format3(data3)

# Step 4: Combine all instruction-response pairs
all_pairs = pairs1 + pairs2 + pairs3

# Step 5: Convert to proper format for training
# Option A: JSONL format
with open("policy_training_data.jsonl_April20", "w") as f:
    for pair in all_pairs:
        f.write(json.dumps(pair) + "\n")

# Option B: Create a Hugging Face Dataset object directly
#train_dataset = Dataset.from_pandas(pd.DataFrame(all_pairs))

# Optional: Split into train and validation sets
#train_val = train_dataset.train_test_split(test_size=0.1)
#train_data = train_val['train']
#val_data = train_val['test']

# Save the processed datasets (optional)
#train_data.save_to_disk("policy_train_dataset")
#val_data.save_to_disk("policy_val_dataset")

In [ ]:
pip install bitsandbytes transformers accelerate peft trl datasets

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineG

In [ ]:
!pip install --upgrade transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 39.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.1
    Uninstalling transformers-4.51.1:
      Successfully uninstalled transformers-4.51.1


In [ ]:
!pip install --upgrade trl

In [ ]:
import os
import torch
from datasets import load_dataset, DatasetDict
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer,SFTConfig

# Configuration
model_id = "meta-llama/Llama-3.2-3B"
output_dir = "/content/drive/MyDrive/Capstone project/policy_analyst_model"

# QLoRA configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
model.config.use_cache = False
model = prepare_model_for_kbit_training(model)

# LoRA configuration
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

# Apply LoRA
model = get_peft_model(model, lora_config)

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Load dataset
raw_data = load_dataset("json", data_files="/content/policy_training_data.jsonl")["train"]

# Combine 'instruction' and 'response' into a new 'text' field
def combine_instruction_response(example):
    example['text'] = example['instruction'] + tokenizer.eos_token + example['response']
    return example

raw_data = raw_data.map(combine_instruction_response)

# Split into train and validation (90% train, 10% validation)
data = raw_data.train_test_split(test_size=0.1, seed=42)


# Training arguments
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    save_steps=100,
    logging_steps=10,
    #evaluation_strategy="steps",
    #eval_steps=100,
    save_total_limit=2,
    fp16=True,
    group_by_length=True,
)

# Initialize trainer
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=data["train"],
    eval_dataset=data["test"],
    #tokenizer=tokenizer,
    peft_config=lora_config,
    #dataset_text_field="text",
    #max_seq_length=2048,
    #packing=True,
)

# Train the model
trainer.train()

# Save the model
trainer.save_model()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Converting train dataset to ChatML:   0%|          | 0/1093 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/1093 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1093 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1093 [00:00<?, ? examples/s]

...

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: shresthabhandari2 (shresthabhandari2-na) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,1.838600
20,1.892700
30,1.859000
40,0.943000
50,1.494100
60,1.048900
70,1.035200
80,1.058700
90,0.672500
100,1.249900
